I am so delighted to share my first kernel with the Kaggle community. I do this for my learning and to mingle myslef with the Data Science community.
Going forward I'll look more into the problems and try to share my ideologies.
Please ask questions and comment below in the comments section regarding the kernel. 


Instead of loading all the libraries at once, I'll import libraries when there is use. So that it can help people understand better.

I use pandas to import and work because it facilitates manipulation of data. It stores the data in dataframes, for easy load and maintenance.

In [ ]:
#importing pandas package

import pandas as pd

In [ ]:
##importing the datasets from local folder

train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

Let's look at the sample data from the train and test dataframes.

In [ ]:
#using head method from the pandas datafram package to read sample data.

train_df.head()

In [ ]:
test_df.head()

From the data above it seems like few fields in the dataframes have NaN (Not a Number, similar to Null)
To find out which fields have such values for certain rows, we use describe method from the same pandas package.

In [ ]:
train_df.describe()

In the above result we can see total rows of Age is less when compared to others. Moreover describe only looked at Numerical fields excluding the categorical fields. With the option include we can involve categorical fields to our description

In [ ]:
train_df.describe(include='all')

In [ ]:
test_df.describe(include='all')

Cabin, Embarked joins the filed Age, which have missing values.
In addition, Cabin and Ticket field has some duplicate values.
Some cabins might be shared by two or more people and some tickets might be couple tickets.

Now let's look at the different type of variables involved in this dataset

1. Categorical : Sex, Cabin, Embarked

2. Numerical : PassengerId, PClass, Age, SibSp. Parch, Fare, Survived

3. Text : Name, Ticket

Let's look at each variable closely and correlate with Survived Field, So that we can pick correct variables for the training purpose.

1. PassengerId : I don't think PassengerId doesn't have any role in predicting Survivial of a passanger.

However let's see how Survival is scattered with PassengerId using matplotlib

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.scatter(train_df['PassengerId'],train_df['Survived'])

plt.xlabel('PassengerId')
plt.ylabel('Survival')
plt.title('Relation between PassengerId and Survival')

You can see that there is no pattern in the PassengerId and Survived, So we are eliminating PassengerId field from our training and test data.

#### PClass and Survived:

PClass is a Ordinal type of Variable, represented Numerically.

It defines the class of the passenger.

Here in this data 

+ Pclass=1 --> UpperClass

+ Pclass=2 --> MiddleClass

+ Pclass=3 --> LowerClass

Plotting Survived and Pclass

Let's see how many people are there in each class

In [ ]:
train_df['Pclass'].value_counts().sort_values()

In [ ]:
pal={0: "red", 1: "green"}
ax = sns.countplot(x = "Pclass", hue="Survived", data = train_df, palette=pal)
plt.title("Passenger class Distribution & Survived vs Not-survived")
plt.xlabel("Pclass");
plt.ylabel("Count of Passengers")
leg = ax.get_legend()
leg.set_title("Passenger class")
legs = leg.texts
legs[0].set_text("Not-Survived")
legs[1].set_text("Survived")
plt.show()

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).sum()

From the data we can say that Most of the Class 3 people couldn't survive from the accident.

+ 136 class 1 survived out of 216 ==> 62.9%

+ 87  class 2 survived out of 184 ==> 47.2%

+ 119 class 3 survived out of 491 ==> 24.2%

#### Inference



From here we can understand that Passenger class has a role to play

The chances of survival depends on the class they are accomodated to.

Better the class, better the chances of the survival.

## Name and Survived:

Generally out of common sense, Death never looks at Name when an accident happens.

Let's look at the data once before eliminating it from the train and test data frame.

In [ ]:
train_df['Name']

-

Wait a second, from the Names I think we could extract some thing.

There is a title for the names 

Let's see whether we have this sort of Titles for all, through that we will try to fill missed Ages.

## Before that we Explore Age and Survived

In [ ]:

g = sns.FacetGrid(train_df, col="Survived", margin_titles=True)
g.map(plt.hist, "Age", color="steelblue", bins=10)

From above graph we can say that People between 20 and 40 survived more compared to others.

The survival rate of Passengers with age group greater than 70 is very minimal

So try to make 4 groups of Ages
+ 0 to 20
+ 20 to 40
+ 40 to 70
+ 70 and above

In [ ]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

Above we tried to observe, different class people, their ages and survival.

Data says that, 
+ Among Class2 people, Children survived more.
+ Class 3 people are much affected with the incident.



## Gender vs Survival

Before looking into the data, I believe that female are more likely to be survived. No Offence

Let's see what happened incase of Titanic disaster.

In [ ]:
pal={0: "red", 1: "green"}
ax = sns.countplot(x = "Sex", hue="Survived", data = train_df, palette=pal)
plt.title("Passenger Gender & Survived vs Not-survived")
plt.xlabel("Pclass");
plt.ylabel("Count of Passengers")
leg = ax.get_legend()
leg.set_title("Survival")
legs = leg.texts
legs[0].set_text("Not-Survived")
legs[1].set_text("Survived")
plt.show()

In [ ]:
train_df[['Sex','Survived']].groupby(['Sex'],as_index=False).mean()

In [ ]:
train_df['Sex'].value_counts()

-

223 females out of 314 => 74% --and-- 109 males out of 577 had survived => 19%.

Let's see hist of class and sex 

In [ ]:

g=sns.FacetGrid(train_df,col='Sex', row='Pclass')
g.map(plt.hist, 'Survived')


In [ ]:
#train_df.loc[(train_df['Sex']=='female') & (train_df['Pclass']==1) & (train_df['Survived']==0)]


Here we can observe 2 things, 
+ If the passenger is male and of class3, he has very minimal chances of survival.
+ Where as if the passenger is female and of class1 or class2, she has very high chances of survival comparative to males.

Overall, More Females are survived from the incident corroborating my assumption :p

## Survival and SibSp and Parch

In [ ]:
train_df[['Survived','SibSp']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
pal={0: "red", 1: "green"}
ax = sns.countplot(x = "SibSp", hue="Survived", data = train_df, palette=pal)
plt.title("Sibsp & Survived vs Not-survived")
plt.xlabel("Count of Siblings and Spouses");
plt.ylabel("Count of Passengers")
leg = ax.get_legend()
leg.set_title("Survival")
legs = leg.texts
legs[0].set_text("Not-Survived")
legs[1].set_text("Survived")
plt.show()

In [ ]:
train_df[['Survived','Parch']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
pal={0: "red", 1: "green"}
ax = sns.countplot(x = "Parch", hue="Survived", data = train_df, palette=pal)
plt.title("Parch & Survived vs Not-survived")
plt.xlabel("Count of Parents and Children");
plt.ylabel("Count of Passengers")
leg = ax.get_legend()
leg.set_title("Survival")
legs = leg.texts
legs[0].set_text("Not-Survived")
legs[1].set_text("Survived")
plt.show()

From above charts we can say that, If a person is alone, he is more prone lo live.

So we will create one field from these two -- alone, not_alone

In [ ]:
train_df['Total_Family_Members']=train_df['Parch']+train_df['SibSp']+1
pal={0: "red", 1: "green"}
ax = sns.countplot(x = "Total_Family_Members", hue="Survived", data = train_df, palette=pal)
plt.title("Family Members & Survived vs Not-survived")
plt.xlabel("Count of Family");
plt.ylabel("Count of Passengers")
leg = ax.get_legend()
leg.set_title("Survival")
legs = leg.texts
legs[0].set_text("Not-Survived")
legs[1].set_text("Survived")
plt.show()

## Ticket and Survival

I'm ignoring Ticket and Cabin, because there are many missing values in both training and test datasets.

## Fare and Survival

In [ ]:
ax=sns.kdeplot(train_df.loc[(train_df['Survived']==0),'Fare'],shade=True,color='r', label='Not Survived')
ax=sns.kdeplot(train_df.loc[(train_df['Survived']==1),'Fare'],shade=True,color='g', label ='Survived')
plt.title('Fare vs Survival')
plt.xlabel('Fare')
plt.ylabel('Survival Density')
#ax=sns.kdeplot(train.loc[(train['Survived'] == 0),'Fare'] 
#ax=sns.kdeplot(train_df.loc[(train_df['Survived']==0),'Age'])

People who boarded the ship with less fare did not survive much, the desnsity plot says

## Fare, Pclass and Survival

In [ ]:
ax=sns.kdeplot(train_df.loc[(train_df['Survived']==0) & (train_df['Pclass']==1),'Fare'],shade=True,color='r', label='Class1')
ax=sns.kdeplot(train_df.loc[(train_df['Survived']==0) & (train_df['Pclass']==2),'Fare'],shade=True,color='g', label='Class2')
ax=sns.kdeplot(train_df.loc[(train_df['Survived']==0) & (train_df['Pclass']==3),'Fare'],shade=True,color='b', label='Class3')

#ax=sns.kdeplot(train_df.loc[(train_df['Survived']==1),'Fare'],shade=True,color='g', label ='Survived')
plt.title('Fare vs Survival')
plt.xlabel('Fare')
plt.ylabel('Survival Density')
#ax=sns.kdeplot(train.loc[(train['Survived'] == 0),'Fare'] 
#ax=sns.kdeplot(train_df.loc[(train_df['Survived']==0),'Age'])

In [ ]:
ax2=sns.kdeplot(train_df.loc[(train_df['Survived']==1) & (train_df['Pclass']==1),'Fare'],shade=True,color='r', label='Class1')
ax2=sns.kdeplot(train_df.loc[(train_df['Survived']==1) & (train_df['Pclass']==2),'Fare'],shade=True,color='g', label='Class2')
ax2=sns.kdeplot(train_df.loc[(train_df['Survived']==1) & (train_df['Pclass']==3),'Fare'],shade=True,color='b', label='Class3')

Seems like there is a good correlation between Fare and the Survival.

People those were aboard with the greater price, survived more.

## Now let's look at the Names and survivals

As discussed we will try to extract the Titles from the Name Field for our further analysis.

In [ ]:
train_df['Name']

In [ ]:
train_df['Title']=train_df['Name'].str.extract(pat= ' ([A-Za-z]+)\.')

## Here we tried to extract words that end with FullStop

In [ ]:
train_df[['Title','Survived']].groupby(['Title'], as_index=False).sum().sort_values(by='Survived',ascending=False)

In [ ]:
train_df['Title'].value_counts()

In [ ]:
train_df[['Title','Survived']].groupby(['Title'], as_index=False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
train_df[['Title','Age']].groupby(['Title'], as_index=False).mean().sort_values(by='Age',ascending=False)

In [ ]:
no_Age = pd.isnull(train_df['Age'])

In [ ]:
no_age_df= train_df[no_Age]

In [ ]:
train_df=train_df.dropna(subset=['Age'])


Here we can see that only Ages of above Titles are been missed, So we will try to input the mean age of these Titles for these NaNs


Dr	42.000000

Mrs	35.898148

Mr	32.368090

Miss	21.773973

Master	4.574167

In [ ]:
A1=no_age_df['Title']=='Dr'
A2=no_age_df['Title']=='Mrs'
A3=no_age_df['Title']=='Mr'
A4=no_age_df['Title']=='Miss'
A5=no_age_df['Title']=='Master'


no_age_df.loc[A1,'Age']=no_age_df.loc[A1,'Age'].fillna(value=42)
no_age_df.loc[A2,'Age']=no_age_df.loc[A2,'Age'].fillna(value=35.89)
no_age_df.loc[A3,'Age']=no_age_df.loc[A3,'Age'].fillna(value=32.4)
no_age_df.loc[A4,'Age']=no_age_df.loc[A4,'Age'].fillna(value=21.8)
no_age_df.loc[A5,'Age']=no_age_df.loc[A5,'Age'].fillna(value=4.6)


In [ ]:
no_age_df.shape

In [ ]:
train_df=train_df.append(no_age_df)

## Embarked and Survival

In [ ]:
pal={0: "red", 1: "green"}
ax = sns.countplot(x = "Embarked", hue="Survived", data = train_df, palette=pal)
plt.title("Embarked & Survived vs Not-survived")
plt.xlabel("Embarkments")
plt.ylabel("Count of Passengers")
leg = ax.get_legend()
leg.set_title("Survival")
legs = leg.texts
legs[0].set_text("Not-Survived")
legs[1].set_text("Survived")
plt.show()

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', height=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

In [ ]:
train_df=train_df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)


Except at Embarkement C, at other places survivals of Female are more.


In [ ]:
train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
train_df['IsAlone'] = 0
train_df.loc[train_df['Total_Family_Members'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp'], axis=1)

In [ ]:
train_df['Embarked'] = train_df['Embarked'].fillna('S')

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:

    train_df.loc[ train_df['Fare'] <= 7.91, 'Fare'] = 0
    train_df.loc[(train_df['Fare'] > 7.91) & (train_df['Fare'] <= 14.454), 'Fare'] = 1
    train_df.loc[(train_df['Fare'] > 14.454) & (train_df['Fare'] <= 31), 'Fare']   = 2
    train_df.loc[ train_df['Fare'] > 31, 'Fare'] = 3
    train_df['Fare'] = train_df['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)

train_df.head(10)

In [ ]:
train_df[['Total_Family_Members', 'Survived']].groupby(['Total_Family_Members'], as_index=False).mean().sort_values(by='Total_Family_Members', ascending=True)

In [ ]:
train_df['Total_Family_Members']=train_df['Total_Family_Members'].astype(str)
FamMapping = { "1": 'Loner', "2": 'Small', "3": 'Small', "4": 'Small',"5": 'Large',"6": 'Large',"7": 'Very Large',"10": 'Very Large'}
train_df['Total_Family_Members'] = train_df['Total_Family_Members'].map(FamMapping)

train_df.head(10)

In [ ]:
train_df

In [ ]:
train = pd.get_dummies(train_df, columns=['Pclass','Sex','Embarked', 'Total_Family_Members', 'Title'], drop_first=False)

In [ ]:
train

In [ ]:
pd.DataFrame(abs(train.corr()['Survived']).sort_values(ascending = False))

In [ ]:

X = train.drop(['Survived'], axis = 1)
y = train["Survived"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = .33, random_state=0)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
headers = X_train.columns 


from sklearn.preprocessing import StandardScaler
st_scale = StandardScaler()


X_train = st_scale.fit_transform(X_train)

X_test = st_scale.transform(X_test)


In [ ]:
pd.DataFrame(X_train).head()

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score


logreg = LogisticRegression(solver='liblinear',
                            penalty= 'l1',random_state = 42   
                            )

logreg.fit(X_train,y_train)

y_pred = logreg.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pd.DataFrame(confusion_matrix(y_test,y_pred),\
            columns=["Predicted Not-Survived", "Predicted Survived"],\
            index=["Not-Survived","Survived"] )

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

https://towardsdatascience.com/understanding-data-science-classification-metrics-in-scikit-learn-in-python-3bc336865019

Go through above article for clear details on sklearn metrics.

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred)

In [ ]:

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
acc_knn

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
acc_gaussian

In [ ]:

perceptron = Perceptron()
perceptron.fit(X_train, y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, y_train) * 100, 2)
acc_linear_svc

In [ ]:

sgd = SGDClassifier()
sgd.fit(X_train, y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, y_train) * 100, 2)
acc_sgd

In [ ]:

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
acc_decision_tree

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

Let's take the test data and preprocess it for predition

In [ ]:
submission = pd.DataFrame({"PassengerId": test_df["PassengerId"], "Survived": Y_pred})
submission.to_csv('../output/submission.csv', index=False)

In [ ]:
test_df=pd.read_csv('test.csv')



In [ ]:
test_df['Title']=test_df['Name'].str.extract(pat= ' ([A-Za-z]+)\.')


In [ ]:
test_df[['Title','Age']].groupby(['Title'], as_index=False).mean().sort_values(by='Age',ascending=False)

In [ ]:
no_Age2 = pd.isnull(test_df['Age'])
no_age_df2= test_df[no_Age]
test_df=test_df.dropna(subset=['Age'])


In [ ]:
A11=no_age_df['Title']=='Dr'
A21=no_age_df['Title']=='Mrs'
A31=no_age_df['Title']=='Mr'
A41=no_age_df['Title']=='Miss'
A51=no_age_df['Title']=='Master'

In [ ]:
no_age_df.loc[A1,'Age']=no_age_df.loc[A1,'Age'].fillna(value=42)
no_age_df.loc[A2,'Age']=no_age_df.loc[A2,'Age'].fillna(value=35.89)
no_age_df.loc[A3,'Age']=no_age_df.loc[A3,'Age'].fillna(value=32.4)
no_age_df.loc[A4,'Age']=no_age_df.loc[A4,'Age'].fillna(value=21.8)
no_age_df.loc[A5,'Age']=no_age_df.loc[A5,'Age'].fillna(value=4.6)

In [ ]:
test_df.describe(include='all')

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]


X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape